<a href="https://colab.research.google.com/github/ManukyanErik/Kaggle/blob/main/Neural%20Network/Dogs_And_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Dogs vs. Cats Redux: Kernels Edition](http:////www.kaggle.com/competitions/dogs-vs-cats-redux-kernels-edition)

# Imports

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

from keras import layers
from keras import models, Model
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras import applications

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import os

In [2]:
import zipfile

zip_files = ['test1', 'train']

for zip_file in zip_files:
    with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/2.Projects/Cat_and_Dogs/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

test1 unzipped
train unzipped


In [3]:
TRAIN_IMAGE_FOLDER_PATH="/content/train"
TRAIN_FILE_NAMES=os.listdir(TRAIN_IMAGE_FOLDER_PATH)

TEST_IMAGE_FOLDER_PATH="/content/test1"
TEST_FILE_NAMES=os.listdir(TEST_IMAGE_FOLDER_PATH)

WIDTH=150
HEIGHT=150

In [4]:
targets=list()
train_full_paths=list()

for train_file_name in TRAIN_FILE_NAMES:
    target=train_file_name.split(".")[0]
    train_full_path=os.path.join(TRAIN_IMAGE_FOLDER_PATH, train_file_name)
    train_full_paths.append(train_full_path)
    targets.append(target)

train_dataset=pd.DataFrame()
train_dataset['train_image_path']=train_full_paths
train_dataset['target']=targets



test_full_paths=list()

for test_file_name in TEST_FILE_NAMES:
    test_full_path=os.path.join(TEST_IMAGE_FOLDER_PATH, test_file_name)
    test_full_paths.append(test_full_path)


test_dataset=pd.DataFrame()
test_dataset['test_image_path']=test_full_paths

In [5]:
target_counts=train_dataset['target'].value_counts()

target_counts

cat    12500
dog    12500
Name: target, dtype: int64

In [6]:
dataset_train, dataset_val=train_test_split(train_dataset, test_size=0.2, random_state=42)

In [7]:
train_datagen=ImageDataGenerator(rotation_range=15,
                                 rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)

train_datagenerator=train_datagen.flow_from_dataframe(dataframe=dataset_train,
                                                      x_col="train_image_path",
                                                      y_col="target",
                                                      target_size=(WIDTH, HEIGHT),
                                                      class_mode="categorical",
                                                      batch_size=150)

Found 20000 validated image filenames belonging to 2 classes.


In [8]:
val_datagen=ImageDataGenerator(rescale=1./255)

val_datagenerator=val_datagen.flow_from_dataframe(dataframe=dataset_val,
                                                   x_col="train_image_path",
                                                   y_col="target",
                                                   target_size=(WIDTH, HEIGHT),
                                                   class_mode="categorical",
                                                   batch_size=150)

Found 5000 validated image filenames belonging to 2 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_dataframe(test_dataset,
                                                  x_col='test_image_path',
                                                  y_col=None,
                                                  class_mode=None,
                                                  shuffle=False,
                                                  target_size=(WIDTH, HEIGHT))

Found 12500 validated image filenames.


# Implementing CNN Architecture with Keras

In [10]:
model=models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape=(WIDTH, HEIGHT, 3)))
model.add(layers.Conv2D(32, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(16, activation="relu"))

model.add(layers.Dense(2, activation="softmax"))

# model.summary()

In [11]:
model.compile(loss="binary_crossentropy", 
             optimizer='adam',
             metrics=["accuracy"])

In [12]:
modelHistory=model.fit(train_datagenerator,
                       epochs=20,
                       validation_data  = val_datagenerator,
                       validation_steps = dataset_val.shape[0]//150,
                       steps_per_epoch  = dataset_val.shape[0]//150)

Epoch 1/20
33/33 [==============================] - 59s 2s/step - loss: 0.6747 - accuracy: 0.6129 - val_loss: 0.7867 - val_accuracy: 0.4937
Epoch 2/20
33/33 [==============================] - 49s 1s/step - loss: 0.6319 - accuracy: 0.6636 - val_loss: 0.9422 - val_accuracy: 0.4952
Epoch 3/20
33/33 [==============================] - 49s 1s/step - loss: 0.6037 - accuracy: 0.6845 - val_loss: 1.1923 - val_accuracy: 0.4935
Epoch 4/20
33/33 [==============================] - 49s 1s/step - loss: 0.5694 - accuracy: 0.7036 - val_loss: 0.8695 - val_accuracy: 0.4929
Epoch 5/20
33/33 [==============================] - 48s 1s/step - loss: 0.5422 - accuracy: 0.7303 - val_loss: 0.7843 - val_accuracy: 0.4933
Epoch 6/20
33/33 [==============================] - 49s 1s/step - loss: 0.5000 - accuracy: 0.7505 - val_loss: 0.8953 - val_accuracy: 0.4952
Epoch 7/20
33/33 [==============================] - 49s 1s/step - loss: 0.4720 - accuracy: 0.7752 - val_loss: 1.2699 - val_accuracy: 0.4970
Epoch 8/20
33/33 [==

# Predict

In [13]:
preds = model.predict(test_generator)

test_dataset['category'] = np.argmax(preds, axis=-1)

In [14]:
output = pd.DataFrame({'id': list(range(1, 12501)),
                       'label': test_dataset['category'] })

output.to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
